In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import ipdb
import time
import sys
import scipy.stats as stt
from itertools import product

import chart_studio.plotly as ply
import chart_studio.tools as plytool
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.offline as plyoff
import plotly.subplots as plysub

# to use plotly offline, need to initialize with a plot\n",
plyoff.init_notebook_mode(connected=True)
init = go.Figure(data=[go.Scatter({'x':[1, 2], 'y':[42, 42], 'mode':'markers'})], layout=go.Layout(title='Init', xaxis={'title':'x'}))
plyoff.iplot(init)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# my imports
sys.path.append('../src/')
from GA.GA import *
from GA.Objective import *
from Utils.Utils import *

In [ ]:
''' define minimization functions & necessary metadata '''
funcs = dict()
funcs['Rastrigin'] = [0, [0, 0], [-5.12, -5.12], [5.12, 5.12]]
funcs['Ackley'] = [0, [0, 0], [-5, -5], [5, 5]]
funcs['Sphere'] = [0, [0, 0, 0, 0], [-10, -10, -10, -10], [10, 10, 10, 10]]
funcs['Rosenbrock'] = [0, [1, 1, 1], [-10, -10, -10], [10, 10, 10]]
funcs['Beale'] = [0, [3, 0.5], [-4.5, -4.5], [4.5, 4.5]]
funcs['Goldstein-price'] = [3, [0, -1], [-2, -2], [2, 2]]
funcs['Booth'] = [0, [1, 3], [-10, -10], [10, 10]]
funcs['Bukin6'] = [0, [-10, 1], [-15, -3], [15, 3]]
funcs['Matyas'] = [0, [0, 0], [-10, -10], [10, 10]]
funcs['Levi13'] = [0, [1, 1], [-10, -10], [10, 10]]
funcs['Himmelblau'] = [0, [3, 2], [-5, -5], [5, 5]] # several minima
funcs['Easom'] = [-1, [np.pi, np.pi], [-100, -100], [100, 100]]
funcs['Styblinsky-Tang'] = [-39.1666*3, [-2.903534, -2.903534, -2.903534], [-5, -5, -5], [5, 5, 5]]

# show
for func in funcs:
    z = MinFunctionValue(func, params=funcs[func][1])[0]
    print('%s Min is %0.2f, Should be %0.2f'%(func, z, funcs[func][0]))

In [ ]:
''' prepare GA input parameters '''
# distribution parameters
bitCnt = 16
func = 'Himmelblau'
lowerB = funcs[func][2]
upperB = funcs[func][3]
bits = [bitCnt]*len(lowerB)

# GA parameters
parmsGA = {'initPerc':0.5, 'showTopRes':10, 'populSize':200, 'numGens':200,
           'noChangeTerm':180, 'convgCrit':0.00001, 'elitism':True, 'mateType':1, 'probXover':0.8,
           'probMutate':0.3, 'probEngineer':0.2, 'optimGoal':-1, 'plotFlag':True, 'printFreq':10,
           'xoverType':1, 'bits':bits, 'lowerB':lowerB, 'upperB':upperB}

# data parameters
parmsData = {'data':None, 'name':func}

# objective parameters
parmsObj = {'function':'MinFunctionValue',
            'arguments':{'data':None, 'params':None, 'func':func}}

In [ ]:
''' run the GA - hold on to your butts '''
# parameters
randSeed = None#42
verb = False
MSims = 1

# init
bestRess = [None]*MSims
bestParams = [None]*MSims
bestScores = [None]*MSims
genBests = [None]*MSims
genBestParams = [None]*MSims
genScores = [None]*MSims
randSeeds = [None]*MSims
timeStamps = [None]*MSims
figGAProgresss = [None]*MSims

for sim in range(MSims):
    print('Executing GA %d of %d'%(sim+1, MSims))
    bestRess[sim], bestParams[sim], bestScores[sim], genBests[sim],\
        genBestParams[sim], genScores[sim], randSeeds[sim], timeStamps[sim],\
        figGAProgresss[sim] = RunGARealOptim(parmsGA, parmsData, parmsObj, verb, randSeed)

# get the overall best
bestIndx = np.argmax(parmsGA['optimGoal']*np.array(bestScores))
bestScore = bestScores[bestIndx]
bestParam = bestParams[bestIndx]
timeStamp = timeStamps[bestIndx]

# best
print('%s function minimized at %r, with value of %0.4f'%(func, bestParam, bestScore))

In [ ]:
''' plot the function, annotating the true and found minima '''
if len(funcs[func][1])==2:
    # get the data range
    n = 100
    mins = funcs[func][-2]
    maxs = funcs[func][-1]
    X = np.linspace(mins[0], maxs[0], n)
    Y = np.linspace(mins[1], maxs[1], n)
    xy = np.array(np.meshgrid(X, Y))
    z = np.zeros(shape=(n*n, 1), dtype=float)

    # compute the values
    xf = xy[0].flatten()
    yf = xy[1].flatten()
    for (indx, (x, y)) in enumerate(zip(xf, yf)):
        parmsObj['arguments']['params'] = [x, y]
        z[indx] = globals()[parmsObj['function']](**parmsObj['arguments'])[0]
    z = z.reshape(n, n)

    # prep the title
    pltTit = '%s Objective Function<br>True Minimum: (%0.3f, %0.3f)= %0.3f<br>GA Minimum: (%0.3f, %0.3f)= %0.3f'%\
        (func, funcs[func][1][0], funcs[func][1][1], funcs[func][0], bestParam[0], bestParam[1], bestScore)

    # plot
    fig = go.Figure(data=[go.Surface(x=xy[0], y=xy[1], z=z)], layout=go.Layout(title=pltTit, width=1000, height=1000))
    fig.update_traces(contours_z=dict(show=True, usecolormap=True, project_z=True))
    anns = list(fig['layout']['annotations'])

    # annotate with the known minimum and GA-found minimum
    knownMinAnn = dict(x=funcs[func][1][0], y=funcs[func][1][1], z=funcs[func][0], showarrow=True, bordercolor="#c7c7c7",
                       borderwidth=2, borderpad=4, bgcolor="green", opacity=0.8, font={'color':'#ffffff'}, align="center",
                       arrowhead=2, arrowsize=1, arrowwidth=2, arrowcolor="#636363", text='True Min: (%0.2f, %0.2f)= %0.2f'%(funcs[func][1][0], funcs[func][1][1], funcs[func][0]))
    anns.append(knownMinAnn)
    GAMinAnn = dict(x=bestParam[0], y=bestParam[1], z=bestScore, showarrow=True, bordercolor="#c7c7c7",
                       borderwidth=2, borderpad=4, bgcolor="blue", opacity=0.8, font={'color':'#ffffff'}, align="center",
                       arrowhead=2, arrowsize=1, arrowwidth=2, arrowcolor="#636363", text='GA Min: (%0.2f, %0.2f)= %0.2f'%(bestParam[0], bestParam[1], bestScore))
    anns.append(GAMinAnn)

    fig.update_layout(scene={'annotations':anns})

    plyoff.plot(fig, filename='../output/GAFuncMinResult_%s_%s.html'%(timeStamp, func), auto_open=True, include_mathjax='cdn')